# An Optimization Problem

In this file I present a problem that I ran into while running quantum circuits on IBM's quantum systems. I do an attempt at generalizing the problem into something a bit more understandable than the quantum computing version. I also present a couple of different techniques and which technique is the best for minimizing instances.

## 1. Problem

Two friends, Alice and Bob, just got into the business of selling apples. The thing is, their apple orchard is half an hour, by foot, from the nearest town: the town of Quantica. The people of Quantica love Alice and Bob's apples, and are constantly making purchases online. Since Alice and Bob just got into the business, they lack proper equipment for apple transportation. At the moment, their best idea is to have Alice stay at the orchard and pick the apples from the apple trees, while Bob travels by foot to Quantica to deliver the apples. Bob has a bin of dimensions $1\times M$ apples, so he can carry at most $M$ apples per trip.

<figure>
    <img class="figure" src="../ipynb/2024-04-20/bin.png" alt="An image of Bob's bin to carry apples.">
    <figcaption><b>Figure 1.</b> A view of the bin Bob uses to carry apples. We see the first four and the last two apples, with the ellipsis indicating additional apples in between.</figcaption>
</figure>

For some reason, Alice and Bob came up with a rule for the people purchasing their apples: the amount of apples a customer buys must be exactly one more than the previous customer, with the counter reseting at the end of the day. So, if the first customer of the day purchases $n$ apples, then the next person must purchase exactly $n+1$ apples, and the next person must purchase exactly $n+2$ apples, and so on. The people of Quantica thought this was a strange rule to implement, but continued to purchase apples nonetheless.

Of course, apples have a very short shelf life. As a result, Alice and Bob have a limit on the maximum number of apples a person can order, call it $m$, with $n \leq m < M$. That is, once a person places an order for $m$ apples, no additional orders can be placed for the remainder of the day. This limit is met daily due to demand from the people of Quantica. So, on any given day Alice and Bob receive orders for $n$, $n+1$, $n+2$, $\ldots$, $m-1$, and $m$ apples -- as $n$ progresses up to $m$.

Alice and Bob's apples are so popular that all of the orders come in almost instantly at the beginning of the day. At first, Bob was walking the orders one by one: first walking to Quantica to take the order with $n$ apples, coming back to the apple orchard, then taking the order with $n+1$ apples, and repeating this process until the last order with $m$ apples. This thing is, this process would result it $m-n+1$ trips to Quantica. That's $m-n+1$ hours!

Alice and Bob thought this was way too much time and have been wanting to talk to someone about their problem. Specifically, they want to know, <u>is there a way to plan their deliveries so that the number of trips Bob makes to Quantica is minimized?</u> Knowing the problem solvers that we are, Alice and Bob have come to us with their problem. How can we help them?

## 2. Solutions

### 2.1 Bottom-Up



In [43]:
def bottom_up(n, m, M):
    # Create a list of orders ranging from n to m inclusive
    orders = list(range(n, m + 1))
    trips = []  # Initialize an empty list to store the trips
    
    # Loop until all orders are processed
    while orders:
        current_trip = []  # Initialize an empty list for the current trip
        remaining_capacity = M  # Set the remaining capacity of the bin to M
        
        # Iterate over the orders in normal order (from smallest to largest)
        for order in orders:
            # If the order can fit in the remaining capacity of the bin
            if order <= remaining_capacity:
                current_trip.append(order)  # Add the order to the current trip
                remaining_capacity -= order  # Decrease the remaining capacity by the order size
        
        # Remove the orders that were added to the current trip from the original orders list
        for order in current_trip:
            orders.remove(order)
        
        # Add the current trip (as a tuple) to the list of trips
        trips.append(tuple(current_trip))
    
    return trips  # Return the list of trips

### 2.2 Top-Down

The biggest order contains $m$ apples. Since $m < M$, it is possible, not necessary, that $\exists\, p\in\mathbb{Z}:$ $m + p = M$, with $n \leq p < m$. If no such $p$ exists, then the order of $m$ apples must be sent by itself. This can be done first, and then we search for some $p_1\in\mathbb{Z}$ s.t. $m - 1 + p_1 = M$, with $n \leq p_1 < m - 1$.

In [25]:
def top_down(n, m, M):
    # Create a list of orders ranging from n to m inclusive
    orders = list(range(n, m + 1))
    trips = []  # Initialize an empty list to store the trips
    
    # Loop until all orders are processed
    while orders:
        current_trip = []  # Initialize an empty list for the current trip
        remaining_capacity = M  # Set the remaining capacity of the bin to M
        
        # Iterate over the orders in reverse (from largest to smallest)
        for order in reversed(orders):
            # If the order can fit in the remaining capacity of the bin
            if order <= remaining_capacity:
                current_trip.append(order)  # Add the order to the current trip
                remaining_capacity -= order  # Decrease the remaining capacity by the order size
        
        # Remove the orders that were added to the current trip from the original orders list
        for order in current_trip:
            orders.remove(order)
        
        # Add the current trip (as a tuple) to the list of trips
        trips.append(tuple(current_trip))
    
    return trips  # Return the list of trips

### 2.3 Combined

In [49]:
def combined_approach(n, m, M):
    orders = list(range(n, m + 1))  # Create a list of orders ranging from n to m inclusive
    trips = []  # Initialize an empty list to store the trips
    
    while orders:
        largest_order = orders.pop()  # Start with the largest order
        current_trip = [largest_order]  # Initialize the current trip with the largest order
        remaining_capacity = M - largest_order  # Set the remaining capacity of the bin

        # Iterate over the orders from smallest to largest
        for order in orders[:]:
            if order <= remaining_capacity:
                current_trip.append(order)  # Add the order to the current trip
                remaining_capacity -= order  # Decrease the remaining capacity by the order size
                orders.remove(order)  # Remove the order from the original list
        
        # Add the current trip (as a tuple) to the list of trips
        trips.append(tuple(current_trip))
    
    return trips  # Return the list of trips

In [61]:
n = 3
m = 100
M = 200

trips_td = top_down(n, m, M)
trips_bu = bottom_up(n, m, M)
trips_combined = combined_approach(n, m, M)

print(f"Total trips required for top-down: {len(trips_td)}")
print("Trips for top-down:", trips_td)

print(f"Total trips required for bottom-up: {len(trips_bu)}")
print("Trips for bottom-up:", trips_bu)

print(f"Total trips required for combined approach: {len(trips_combined)}")
print("Trips for combined approach:", trips_combined)

Total trips required for top-down: 26
Trips for top-down: [(100, 99), (98, 97, 5), (96, 95, 9), (94, 93, 13), (92, 91, 17), (90, 89, 21), (88, 87, 25), (86, 85, 29), (84, 83, 33), (82, 81, 37), (80, 79, 41), (78, 77, 45), (76, 75, 49), (74, 73, 53), (72, 71, 57), (70, 69, 61), (68, 67, 65), (66, 64, 63, 7), (62, 60, 59, 19), (58, 56, 55, 31), (54, 52, 51, 43), (50, 48, 47, 46, 8), (44, 42, 40, 39, 35), (38, 36, 34, 32, 30, 28), (27, 26, 24, 23, 22, 20, 18, 16, 15, 6, 3), (14, 12, 11, 10, 4)]
Total trips required for bottom-up: 30
Trips for bottom-up: [(3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19), (20, 21, 22, 23, 24, 25, 26, 27), (28, 29, 30, 31, 32, 33), (34, 35, 36, 37, 38), (39, 40, 41, 42), (43, 44, 45, 46), (47, 48, 49, 50), (51, 52, 53), (54, 55, 56), (57, 58, 59), (60, 61, 62), (63, 64, 65), (66, 67), (68, 69), (70, 71), (72, 73), (74, 75), (76, 77), (78, 79), (80, 81), (82, 83), (84, 85), (86, 87), (88, 89), (90, 91), (92, 93), (94, 95), (96, 97), (98, 99), (10